In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

GHCND_PATH = Path("/mnt/d/climate_data/ghcnd/data/")
HPD_PATH = Path("/mnt/d/climate_data/hpd/data/")
DATA_DOC_PATH = Path("./data/dataset_docs/")

OE_PATH = Path("/mnt/d/climate_data/HPD_CONUS_OEVENTS/")
OE_PATH.mkdir(parents=True, exist_ok=True)

In [2]:
def load_stn_tmean(stn_id):
    """
    Load GHCN-d TMEAN data that passed QC into a dataframe with datetime index
    """
    # Load and reindex data
    df = pd.read_csv(GHCND_PATH / f"{stn_id}.csv", low_memory=False, usecols=['DATE', 'TMAX', 'TMIN', 'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES'], index_col='DATE', parse_dates=True, na_values=[9999, -9999]).asfreq('D')

    # Remove flagged data
    tmax_split = df['TMAX_ATTRIBUTES'].str.split(',', expand=True)
    tmin_split = df['TMIN_ATTRIBUTES'].str.split(',', expand=True)

    ## Set to NaN if flag1 or flag2 is not empty
    df.loc[(tmax_split[0] != '') | (tmax_split[1] != ''), 'TMAX'] = np.nan
    df.loc[(tmin_split[0] != '') | (tmin_split[1] != ''), 'TMIN'] = np.nan

    df["TMEAN"] = (df["TMAX"] + df["TMIN"]) / 20 # TEMP data stored as tenth of Celsius

    return df.drop(columns=['TMAX', 'TMIN', 'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES'])


def load_hpd_data(stn_id):
    """
    Load HPD data into a Pandas DataFrame with datetime index, making sure all observation inside flagged accumulation period are removed
    """
    hourly_cols = [f"HR{i:02d}Val" for i in range(24)]
    MF_cols = [f"HR{i:02d}MF" for i in range(24)]
    cols =['DATE', *hourly_cols, *MF_cols]

    # Load and reindex data to identify missing days
    df = pd.read_csv(HPD_PATH / f"{stn_id}.csv", low_memory=False, na_values=[9999, -9999], usecols=cols, index_col='DATE', parse_dates=True).asfreq('D')

    prcp = df[hourly_cols].values.flatten() * 0.254 # Hundredth of inch --> mm
    flags = df[MF_cols].values.flatten()

    # # Remove all non-missing observation inside flagged accumulation period 
    #len_before = len(prcp[~np.isnan(prcp)])
    accum_mask = np.isin(flags, ['a', '.', 'A']).astype(bool)
    prcp[accum_mask] = np.nan
    #print(f"{len_before - len(prcp[~np.isnan(prcp)])} suspicious data removed")

    # Time index:
    repeated_dates = np.repeat(df.index, 24)
    hours = np.tile(np.arange(24), len(df))
    index = repeated_dates + pd.to_timedelta(hours, unit='h')
    return pd.DataFrame({'PRCP': prcp}, index=index)


def count_valid_years(df, perc=0.3):
    """
    Count year with less than perc365 missing values
    """
    min_valid = 365 * (1 - perc)
    yearly_counts = df.resample('YE').count()
    
    if len(df.columns) > 1:
        valid_years = (yearly_counts.iloc[:, 0] >= min_valid).sum()
    else:
        valid_years = (yearly_counts.iloc[:, 0] >= min_valid).sum()
    return valid_years

In [3]:
# HPD Stations
hpd_stations = (
    pd.read_csv(DATA_DOC_PATH / "HPD_v02r02_stationinv_c20240502.csv")
    .drop(columns=['Last_Half_POR', 'PCT_Last_Half_Good', 'Last_Qtr_POR', 'PCT_Last_Qtr_Good', 'Sample_Interval (min)', 'WMO_ID'])
)
hpd_stations['PCT_POR_Good'] = hpd_stations['PCT_POR_Good'].str.rstrip('%').astype(float)
hpd_stations['Num_Years'] = (hpd_stations['POR_Date_Range'].str.slice(9, 13).astype(int) -
                              hpd_stations['POR_Date_Range'].str.slice(0, 4).astype(int))

ghcnd_stations = pd.read_csv(DATA_DOC_PATH / "GHCND_TMEAN_station_info.csv")
ghcnd_stations = ghcnd_stations[ghcnd_stations["Num_Years"] > 0]
pt_stations = hpd_stations.merge(ghcnd_stations.drop(columns=['LAT', 'LON', 'ELEV', 'STATE', 'NAME']), on='StnID', how='inner', suffixes=('_P', '_T'))
pt_stations

,StnID,Lat,Lon,Elev,State/Province,Name,UTC_Offset,POR_Date_Range_P,PCT_POR_Good_P,Num_Years_P,POR_Date_Range_T,Num_Years_T,PCT_POR_Good_T
0,USC00010063,34.2110,-87.1784,239.6,AL,ADDISON,-6,19480601-20240416,84.6,76,19380301-20240503,87.0,19.5
1,USC00010252,31.3071,-86.5226,76.2,AL,ANDALUSIA 3 W,-6,19800301-20180205,89.1,38,19121005-20180131,107.0,70.6
2,USC00010369,33.2941,-85.7788,311.5,AL,ASHLAND 3 ENE,-6,19480601-20130804,84.6,65,19400301-20130731,74.0,71.9
3,USC00010390,34.7752,-86.9508,210.0,AL,ATHENS,-6,19820601-20240304,84.5,42,19410520-20240504,84.0,38.4
4,USC00010402,31.1820,-87.4390,91.4,AL,ATMORE,-6,19480601-20240401,64.4,76,19400327-20240430,85.0,73.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,USC00518838,21.4992,-158.0111,306.6,HI,UPPER WAHIAWA 874.3,-10,20150110-20240422,80.1,9,19710401-20150430,45.0,29.1
1435,USC00518941,21.8967,-159.5569,65.5,HI,WAHIAWA 930,-10,19650301-20170410,87.5,52,19491001-20240329,76.0,8.0
1436,USC00518945,21.4967,-158.0497,260.3,HI,WAHIAWA DAM 863,-10,19650419-20240422,94.7,59,19400101-19780430,39.0,1.9
1437,USC00519195,21.5747,-158.1204,3.0,HI,WAIALUA 847,-10,19650301-20240311,89.3,59,19080101-20011130,94.0,67.3


In [4]:
for idx, row in pt_stations.iterrows():
    stn_id = row["StnID"]

    tmean_df = load_stn_tmean(stn_id)
    prcp_df = load_hpd_data(stn_id)
    prcp_df["DATE"] = (
        prcp_df.index.normalize()
    ) 
    merged = (
        prcp_df.merge(tmean_df, how="left", on="DATE") # Match prcp with daily temperature on the day of the event
        .drop(columns=["DATE"])
    )
    merged.index = prcp_df.index
    merged = merged.dropna()

    num_year_overlap = len(merged.index.year.unique())
    effective_num_year = count_valid_years(merged, perc=0.3)

    pt_stations.loc[pt_stations["StnID"] == stn_id, "Num_Year_Overlap"] = len(merged.index.year.unique())
    pt_stations.loc[pt_stations["StnID"] == stn_id, "Num_Valid_Years"] = count_valid_years(merged, perc=0.3)

In [6]:
potential_stations = pt_stations.query("24.5 < Lat < 49.5 and -125 < Lon < -66.75 and Num_Valid_Years >= 30").sort_values(['Num_Valid_Years'],ascending=False).reset_index(drop=True)
potential_stations

,StnID,Lat,Lon,Elev,State/Province,Name,UTC_Offset,POR_Date_Range_P,PCT_POR_Good_P,Num_Years_P,POR_Date_Range_T,Num_Years_T,PCT_POR_Good_T,Num_Year_Overlap,Num_Valid_Years
0,USC00241737,47.8205,-112.1921,1172.0,MT,CHOTEAU,-7,19400101-20240401,93.8,84,18930101-20240504,132.0,83.6,85.0,85.0
1,USC00411017,31.1445,-99.3492,521.5,TX,BRADY,-6,19400501-20240401,93.3,84,18930101-20240331,132.0,67.3,84.0,84.0
2,USC00085612,28.1136,-80.6540,10.7,FL,MELBOURNE WFO,-5,19420101-20240401,95.0,82,19370721-20240503,88.0,94.8,83.0,83.0
3,USW00093914,31.7831,-95.6039,141.7,TX,PALESTINE 2 NE,-6,19400201-20240213,86.7,84,19300101-20240324,95.0,92.2,83.0,83.0
4,USC00087851,28.3381,-82.2602,52.7,FL,SAINT LEO,-5,19440817-20240403,96.6,80,18950301-20240430,130.0,98.6,81.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,USC00106250,42.0264,-115.3596,1570.3,ID,MURPHY (DESERT) HOT SPRINGS,-7,19871209-20230603,67.1,36,19871203-20230630,37.0,68.8,31.0,30.0
861,USC00201361,43.5861,-83.1805,212.8,MI,CASS CITY 1 SSW,-5,19921103-20240402,89.9,32,19590501-20240503,66.0,45.0,30.0,30.0
862,USC00317845,35.3110,-81.5710,249.9,NC,SHELBY 2 NW,-5,19940901-20240331,93.2,30,18930208-20240504,132.0,65.3,31.0,30.0
863,USC00115825,41.3713,-88.4332,159.7,IL,MORRIS 1NW,-6,19951101-20240401,86.5,29,19111201-20240504,114.0,76.1,30.0,30.0


In [7]:
potential_stations.to_csv(DATA_DOC_PATH / "potential_pt_stations.csv", index=False)

In [ ]:
#potential_stations = pd.read_csv(DATA_DOC_PATH / "hpd_hadisd.csv")

# Compute Ordinary Events

In [2]:
from numpy.lib.stride_tricks import sliding_window_view

In [3]:
def load_hpd_data(stn_id):
    """
    Load HPD data into a Pandas DataFrame with datetime index, making sure all observation inside flagged accumulation period are removed
    """
    hourly_cols = [f"HR{i:02d}Val" for i in range(24)]
    MF_cols = [f"HR{i:02d}MF" for i in range(24)]
    cols =['DATE', *hourly_cols, *MF_cols]

    # Load and reindex data to identify missing days
    df = pd.read_csv(HPD_PATH / f"{stn_id}.csv", low_memory=False, na_values=[9999, -9999], usecols=cols, index_col='DATE', parse_dates=True).asfreq('D')

    prcp = df[hourly_cols].values.flatten() * 0.254 # Hundredth of inch --> mm
    flags = df[MF_cols].values.flatten()

    # # Remove all non-missing observation inside flagged accumulation period 
    #len_before = len(prcp[~np.isnan(prcp)])
    accum_mask = np.isin(flags, ['a', '.', 'A']).astype(bool)
    prcp[accum_mask] = np.nan
    #print(f"{len_before - len(prcp[~np.isnan(prcp)])} suspicious data removed")

    # Time index:
    repeated_dates = np.repeat(df.index, 24)
    hours = np.tile(np.arange(24), len(df))
    index = repeated_dates + pd.to_timedelta(hours, unit='h')
    return pd.DataFrame({'PRCP': prcp}, index=index)

def ordinary_events(data, timeindex, durations, delta_t, thresh=0.254, sep_len="24h"):
    """ 
    Split data into storms
    data: 1 for precipitation data and 0 for dry period
    Note: 0s are not removed at the begining and at the end if they are not longer than sep_len. Needs modification if interested in storm duration.
    Args:
    ----
    delta_t: Time intervals between samples (Pandas frequency format e.g. "24H")
    """

    
    data = np.array(data)

    if not isinstance(durations, list):
        durations = [durations]

    # Find where 0 runs start and their lengths
    is_zero = (data < thresh).astype(int)
    changes = np.diff(is_zero.astype(int), prepend=0, append=0)

    zero_starts = np.where(changes == 1)[0]
    zero_ends = np.where(changes == -1)[0]
    zero_lengths = zero_ends - zero_starts

    # Indices of zero runs that are at least sep_len long
    sep_len_su = int(pd.Timedelta(sep_len) / pd.Timedelta(delta_t)) # sep_len in sample units
    long_zero_spans = [(start, end) for start, end, length in zip(zero_starts, zero_ends, zero_lengths) if length >= sep_len_su]

    # Slice the data
    storms = []
    storms_time = []
    prev_end = 0
    for start, end in long_zero_spans:
        if prev_end < start:
            storms.append(data[prev_end:start])
            storms_time.append(timeindex[prev_end:start])
        prev_end = end
    if prev_end < len(data):
        storms.append(data[prev_end:])
        storms_time.append(timeindex[prev_end:])
    #return [arr for arr in storms if len(arr) > 0 and not np.isnan(arr).all()] # If not missing

    # Identify events
    ## Compute window size from duration
    window_sizes = [int(pd.Timedelta(d) / pd.Timedelta(delta_t)) for d in durations]
    res = []
    for ws in window_sizes:
        events = {'datetime': [], 'PRCP': []}
        for jj, storm in enumerate(storms):
            if len(storm) >= ws and not np.isnan(storm).all(): # If not missing
                if ws == 1:
                    max_idx = np.nanargmax(storm)
                    storm_max = storm[max_idx]
                    event_time = storms_time[jj][max_idx]
                else:
                    windows = sliding_window_view(storm, ws) # safer than np.lib.stride_tricks.as_strided
                    agg_values = np.nansum(windows, axis=1)
                    max_idx = agg_values.argmax()
                    storm_max = agg_values[max_idx]
                    event_time = storms_time[jj][max_idx + ws - 1]

                events["datetime"].append(event_time)
                events["PRCP"].append(storm_max)
            else:
                continue
            
        res.append(pd.DataFrame(events).set_index("datetime"))
    return res


def compute_ordinary_events_from_hpd(stations, durations):
    duration_dirs = {d: (OE_PATH / d) for d in durations}
    for path in duration_dirs.values():
        path.mkdir(parents=True, exist_ok=True)

    for stn_id in stations:
        prcp_df = load_hpd_data(stn_id)
        oes_list = ordinary_events(
            prcp_df["PRCP"].values,
            prcp_df.index.values,
            durations=durations,
            delta_t="1h",
            thresh=0.254,
            sep_len="24h",
        )
        for d, df in zip(durations, oes_list):
            df.to_csv(duration_dirs[d] / f"{stn_id}.csv")
    

In [6]:
stations = potential_stations["StnID"]
durations = ["1h", "24h"]
compute_ordinary_events_from_hpd(stations, durations)